In [21]:
using GLMakie
using Revise
include("EFITModule/EFIT.jl")


Main.EFIT

# Initialize files

In [23]:
bovs = Vector{String}()
dir = "BOVTests/"
for f in readdir(dir)
    fname, ftype = split(f,".")
    if ftype == "bov"
        push!(bovs,"$dir/$f")
    end
end
data = Main.EFIT.readBOV(bovs[1]);
print(size(data))

(434, 358, 716)

# Volumetric animation

In [3]:
data = Main.EFIT.readBOV(bovs[1]);
fig,ax,plt = volume(data,algorithm=:mip,colorrange = (0, 0.01),colormap=:curl, transparency=true)
record(fig, "Animation_Volume.mp4", bovs; framerate = 30) do path
    Main.EFIT.readBOV!(data, path)
    plt.volume = data
end

SystemError: SystemError: opening file "mediadata2022 DataRailTest_2,5MHz_3_May/DataDownsampled-10.bin": No such file or directory

# 2D Slices

## X Slice

In [4]:
depth = 217
data = Main.EFIT.readBOV(bovs[1]);
fig,ax,plt = heatmap(data[depth,:,:],colorrange = (0, 0.01),colormap=:curl)
record(fig, "Animation_XSlice.mp4", bovs; framerate = 30) do path
    Main.EFIT.readBOV!(data, path)
    plt.heatmap = data[depth,:,:]
end

"Animation_XSlice.mp4"

# Composite

In [28]:
xDepth = 217
yDepth = 177
zDepth = 358

data = Main.EFIT.readBOV(bovs[1]);

fig,ax,plt = volume(data,algorithm=:mip,colorrange = (0, 0.01),colormap=:curl, transparency=true)

record(fig, "Animation_Composite.mp4", bovs; framerate = 30) do path
    Main.EFIT.readBOV!(data, path)
    plt.data = data

end

"Animation_Composite.mp4"